A tester sur ESP32 avec analyseur de trame

[ESP32 pins-and-gpio](https://docs.micropython.org/en/latest/esp32/quickref.html#pins-and-gpio){target=_blank}
[ESP32 delay-and-timing](https://docs.micropython.org/en/latest/esp32/quickref.html#delay-and-timing){target=_blank}

In [ ]:
# Programme de test de connexion de la sortie 18 en blink

import esp32
from machine import Pin
import time

pin18 = Pin(18, Pin.OUT)

print('Let\'s GO')

while True :
    pin18.on()
    time.sleep_ms(30)
    pin18.off()
    time.sleep_ms(20)



[ESP32 rmt](https://docs.micropython.org/en/latest/library/esp32.html#rmt){target=_blank}

In [2]:
frequence = 80e6
periode = 1/frequence
periode

1.25e-08

In [ ]:
import esp32
from machine import Pin

r = esp32.RMT(2, pin=Pin(18), clock_div=80) # 80 pour avoir 80/80000000 = 1 µs
print(r.source_freq()) # 80000000 Hz

signal=(88,8,4,44,40,8,40,8,40,8,40) # les durées sont des multiples de 4 x 1 µs 
r.write_pulses(signal, start=0) # commence par l'état 0

In [2]:
trameDMXdec = [150,20,0,0,255,0,0,0,0,0,0,0,0,0,0,0] # les valeurs de 16 canaux

In [5]:
# Conception trameDMXstr chaine de caractères
le_break = '0'*22 # 22 états 0
le_mab = '11' # Mark After Break
code_depart = '0' + '0'*8 + '11' # 1 bit de start à 0 + une valeur = 0 en binaire sur huit bits + 2 bits de stop à 1
trameDMXstr = le_break + le_mab + code_depart

for i in range(0, len(trameDMXdec)) :
    trameX = '0' # bit de start
    for masque in [1,2,4,8,16,32,64,128] :
        if (trameDMXdec[i] & masque != 0) :
            trameX += '1'
        else :
            trameX += '0'
    trameX += '11' # 2 bits de stop
    trameDMXstr += trameX
    
print(trameDMXstr)
print(len(trameDMXstr))

0000000000000000000000110000000001100110100111000101000110000000001100000000011011111111110000000001100000000011000000000110000000001100000000011000000000110000000001100000000011000000000110000000001100000000011
211


In [6]:
# Conception trameRMT à partir du découpage de trameDMXstr
trameRMT = [] # liste des durées Etats haut puis Etats bas
nb = 0
lecture_precedente = '0'

for i in range (0, len(trameDMXstr)) :
    if trameDMXstr[i] == lecture_precedente :
        nb += 4 # 4µs pour 250kHz
    else :
        trameRMT.append(nb)
        lecture_precedente = trameDMXstr[i]
        nb = 4
trameRMT.append(nb)
print(trameRMT)


[88, 8, 36, 8, 8, 8, 4, 4, 8, 12, 12, 4, 4, 4, 12, 8, 36, 8, 36, 8, 4, 40, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8]


In [ ]:
#Christian conceptiontrameRMTtest512RMT2COMPLET.py

import esp32
from machine import Pin
import time
 
r = esp32.RMT(2, pin=Pin(18), clock_div=80)
print(r.source_freq())
# The channel resolution is 1us (1/(source_freq/clock_div))

#Trame DMX
# TrameDMXdix=[0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,[88, 8, 36, 8, 8, 4, 4, 28, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8]0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
TrameDMXdix=[255,0,255,0,0,0,0,0,0,0,0,0,0,0,0,0]#1

print(TrameDMXdix)  #test taille trameDMX          

#Conception trameDMXstr 
Break='0000000000000000000000'#88ms
MAB='11' # Mark After Break
code_depart ='00000000011'#1bit start + value=0 + 2bits stop
trameDMXstr=Break+MAB+code_depart
for i in range (0, len(TrameDMXdix)):
    trameX='0'#start
    for masque in [1,2,4,8,16,32,64,128]:
       if (TrameDMXdix[i]&masque != 0) :
           trameX=trameX+'1'
       else :
           trameX=trameX+'0'
    trameX=trameX+'11'# 2 bits stop
    trameDMXstr=trameDMXstr+trameX
print(trameDMXstr)
print(len(trameDMXstr))#calcul-26-11 = -37

#Conception trameRMT à partir du découpage de trameDMXstr
trameRMT=[] # liste des durées Etats haut puis Etats bas 
nb=0
lectureprecedente='0'
# nbmot=len(trameDMXstr)
for i in range (0,len(trameDMXstr)):
    if trameDMXstr[i]==lectureprecedente:
        nb=nb+4 # 4us = 250kHz
    else :
        trameRMT.append(nb)
        lectureprecedente=trameDMXstr[i]
        nb=4 #4us = 250kHz
trameRMT.append(nb)
print (trameRMT)         

#conception tuple RMT
signal=trameRMT
signal_Rouge=(88, 8, 36, 8, 4, 40, 4, 40, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8) 
signal_Vert=(88, 8, 36, 8, 4, 40, 36, 8, 4, 40, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8)
#RS432
while(True):
    start = time.ticks_ms() # get millisecond counter
    delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    while (delta<=5000) : 
        r.write_pulses(signal_Rouge, 0)
        delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    start = time.ticks_ms() # get millisecond counter
    delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    while (delta<=5000) : 
        r.write_pulses(signal_Vert, 0)
        delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    


In [ ]:
#Christian  conceptiontrameRMTtest512RMT2COMPLETbicolor.py

import esp32
from machine import Pin
import time
 
r = esp32.RMT(2, pin=Pin(18), clock_div=80)
print(r.source_freq())
# The channel resolution is 1us (1/(source_freq/clock_div))

#Trame DMX
# TrameDMXdix=[0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
#              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
TrameDMXdix=[255,0,255,0,0,0,0,0,0,0,0,0,0,0,0,0]#1

print(TrameDMXdix)  #test taille trameDMX          

#Conception trameDMXstr 
Break='0000000000000000000000'#88ms
MAB='11' # Mark After Break
code_depart ='00000000011'#1bit start + value=0 + 2bits stop
trameDMXstr=Break+MAB+code_depart
for i in range (0, len(TrameDMXdix)):
    trameX='0'#start
    for masque in [1,2,4,8,16,32,64,128]:
       if (TrameDMXdix[i]&masque != 0) :
           trameX=trameX+'1'
       else :
           trameX=trameX+'0'
    trameX=trameX+'11'# 2 bits stop
    trameDMXstr=trameDMXstr+trameX
print(trameDMXstr)
print(len(trameDMXstr))#calcul-26-11 = -37

#Conception trameRMT à partir du découpage de trameDMXstr
trameRMT=[] # liste des durées Etats haut puis Etats bas 
nb=0
lectureprecedente='0'
# nbmot=len(trameDMXstr)
for i in range (0,len(trameDMXstr)):
    if trameDMXstr[i]==lectureprecedente:
        nb=nb+4 # 4us = 250kHz
    else :
        trameRMT.append(nb)
        lectureprecedente=trameDMXstr[i]
        nb=4 #4us = 250kHz
trameRMT.append(nb)
print (trameRMT)         

#conception tuple RMT
signal=trameRMT
signal_Rouge=(88, 8, 36, 8, 4, 40, 4, 40, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8) 
signal_Vert=(88, 8, 36, 8, 4, 40, 36, 8, 4, 40, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8, 36, 8)
#RS432
while(True):
    start = time.ticks_ms() # get millisecond counter
    delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    while (delta<=5000) : 
        r.write_pulses(signal_Rouge, 0)
        delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    start = time.ticks_ms() # get millisecond counter
    delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    while (delta<=5000) : 
        r.write_pulses(signal_Vert, 0)
        delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
    


In [ ]:
#Christian  conceptiontrameRMTtest512RMTVFINALE.py

#Trame DMX
#TrameDMXdix=[255,20,125,74,13,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0]#1
TrameDMXdix=[150,20,0,0,255,0,0,0,0,0,0,0,0,0,0,0]#1

print(TrameDMXdix)  #test taille trameDMX          

#Conception trameDMXstr 
Break='0000000000000000000000'#88ms
MAB='11' # Mark After Break
code_depart ='00000000011'#1bit start + value=0 + 2bits stop
trameDMXstr=Break+MAB+code_depart
for i in range (0, len(TrameDMXdix)):
    trameX='0'#start
    for masque in [1,2,4,8,16,32,64,128]:
       if (TrameDMXdix[i]&masque != 0) :
           trameX=trameX+'1'
       else :
           trameX=trameX+'0'
    trameX=trameX+'11'# 2 bits stop
    trameDMXstr=trameDMXstr+trameX
print(trameDMXstr)
print(len(trameDMXstr))#calcul-26-11 = -37

#Conception trameRMT à partir du découpage de trameDMXstr
trameRMT=[] # liste des durées Etats haut puis Etats bas 
nb=0
lectureprecedente='0'
# nbmot=len(trameDMXstr)
for i in range (0,len(trameDMXstr)):
    if trameDMXstr[i]==lectureprecedente:
        nb=nb+4 # 4us = 250kHz
    else :
        trameRMT.append(nb)
        lectureprecedente=trameDMXstr[i]
        nb=4 #4us = 250kHz
trameRMT.append(nb)
print (trameRMT)         

